## BPM-scanner: Scans a directory of mp3 files, returns a table of files and beats-per-minute

##### Note: requires the pydub and ffmpeg packages to read mp3s, which can be installed with:

<i>\> conda install -c conda-forge pydub</i><br>
<i>\> conda install -c conda-forge ffmpeg</i>

In [248]:
import numpy as np
from scipy.signal import correlate
from os import listdir
import pydub
import pprint

class parameters():
    def __init__(self):
        self.dir_name = "C:\\mp3\\"  # Directory with mp3 files to scan
        self.min_bpm = 75            # Minimum BPM limit of scan
        self.max_bpm = 150           # Maximum BPM limit of scan
        self.num_subsample = 31      # number of subsamples per song
        self.len_subsample = 20.0    # length of subsample in seconds; must be shorter than song
        self.verbose = False         # print debug info
        
def read_directory(params):
    
    directory = listdir(params.dir_name)
    mp3_list = [filename for filename in directory if filename[-4:] == ".mp3"]   
    
    return mp3_list


def process_mp3s(params, mp3_list):
    
    bpm_array = []

    for i in range(len(mp3_list)):
        
        mp3_path = params.dir_name + mp3_list[i]
        print(f"Processing {mp3_path} ", end = '')
        
        segment = pydub.AudioSegment.from_mp3(mp3_path)
        bpm, stdev = get_bpm(params, segment)
        
        if (stdev < 0.01):
            confidence = '+++++'
        elif (stdev < 0.1):
            confidence = '++++'
        elif (stdev < 1.0):
            confidence = '+++'
        elif (stdev < 10.0):
            confidence = '++'
        else:    
            confidence = '+'
        
        bpm_array.append([mp3_list[i], bpm, confidence])

    return bpm_array
    
def process_audio(params, segment):
    
    # if stereo, average to a mono signal
    if (segment.channels == 2):
        raw = average_channels(segment) 
    
    else:
        raw = np.array(segment.get_array_of_samples())
    
    # signed 16-bit ints -> [-1 to +1]
    normalized = raw / 32768.0
    
    return normalized
    
def get_bpm(params, segment):

    normalized = process_audio(params, segment)
    rate = segment.frame_rate
    print("...")
    
    bpm_samples = []
    if(params.verbose == True):
        print(f"BPM samples: ", end = '')
    
    for i in range(params.num_subsample):
        subsample = grab_subsample(params, normalized, rate, i)
        max_corr_index = get_max_correlation(params, subsample, rate)
        
        # BPM = 60 sec / seconds_between_beats = 60 sec / (max_correlated_datapoint / data_per_sec)
        bpm = 60.0 / (max_corr_index / rate)
        
        # the max correlation may come 2 or 4 or 8 measures out, producing very low bpm numbers
        # so double until we're above the minimum bpm value
        while(bpm < params.min_bpm):
            bpm *= 2
        
        bpm_samples.append(bpm)
        
        if(params.verbose == True):
            print(f"{bpm:.3f} ", end = '')
    
    bpm_samples.sort()
    
    # remove outliers of outer 1/4 on each side and calc variance
    stdev = np.std(bpm_samples[int(0.25 * params.num_subsample):int(0.75 * params.num_subsample)])
    
    if(params.verbose == True):
        print(f"Stdev: {stdev:.4f}")
        
    # report median
    bpm = bpm_samples[int(params.num_subsample/2)]
    
    return bpm, stdev

def average_channels(segment):
    
    left_channel, right_channel = segment.split_to_mono()
    
    left_raw = np.array(left_channel.get_array_of_samples())
    right_raw = np.array(right_channel.get_array_of_samples())
    
    averaged = (left_raw + right_raw) / 2.0
    
    #averaged = np.array([(normalized[2*i] + normalized[2*i+1])/2.0 for i in range(len(normalized)//2)])
        
    return averaged

def grab_subsample(params, normalized, rate, i):
    
    total_datapoints = int(params.len_subsample * rate)
    last_valid_subsample_start = len(normalized) - total_datapoints
    
    # often very little rhythm happens at the very beginning and very end of songs,
    # so subdivide the song equally into (num_subsample + 2) parts and discard the outer two
    
    start_location = int((i + 1) * (last_valid_subsample_start / (params.num_subsample + 2)))
    
    subsample = normalized[start_location:(start_location + total_datapoints)]
    
    return subsample


def get_max_correlation(params, normalized, rate):
    
    # Run autocorrelation
    correlation = correlate(normalized, normalized)
    
    # find midpoint of correlation = zero offset of auto-correlation
    half_len = len(correlation)//2

    # grab the audio offset from t_start to the end, otherwise zero offset produces the max correlation value
    t_start = 1.0 / (params.max_bpm / 60.0)

    corr_range = correlation[half_len + int(t_start * rate):]
    
    # Find maximum auto-correlation value
    max_correlation = np.argmax(corr_range) + int(t_start * rate)
    
    return max_correlation

In [249]:
import pandas as pd

free_params = parameters()

mp3_list = read_directory(free_params)

bpm_array = process_mp3s(free_params, mp3_list)

pd.DataFrame(bpm_array, columns = ["Song Title", "BPM", "Confidence"])


Processing C:\mp3\Bee Gees - Stayin Alive.mp3 ...
Processing C:\mp3\Black V Neck - Electric.mp3 ...
Processing C:\mp3\Cheryl Lynn - Got To Be Real.mp3 ...
Processing C:\mp3\Daft Punk - Homework - 09 - Teachers.mp3 ...
Processing C:\mp3\Erasure - Always.mp3 ...
Processing C:\mp3\Fats Waller - All That Meat and No Potatoes.mp3 ...
Processing C:\mp3\Ice Cube - It Was A Good Day.mp3 ...
Processing C:\mp3\J. S. Bach - Glenn Gould - Prelude No. 1 C Major BWV 846.mp3 ...
Processing C:\mp3\John Cameron - Liquid Sunshine.mp3 ...
Processing C:\mp3\Johnny Cash - Ring of Fire.mp3 ...
Processing C:\mp3\Justice - Genesis.mp3 ...
Processing C:\mp3\Kartell - La Jeunesse Retrouvee.mp3 ...
Processing C:\mp3\Kool and The Gang - Get Down on it.mp3 ...
Processing C:\mp3\Kool Keith - Dr. Octagon - Earth People.mp3 ...
Processing C:\mp3\Lou Reed - Perfect Day.mp3 ...
Processing C:\mp3\Men at Work - Land Down Under.mp3 ...
Processing C:\mp3\Mozart - Requiem - Agnus Dei.mp3 ...
Processing C:\mp3\New Order - Bl

,Song Title,BPM,Confidence
0,Bee Gees - Stayin Alive.mp3,103.595601,+++++
1,Black V Neck - Electric.mp3,125.000000,+++++
2,Cheryl Lynn - Got To Be Real.mp3,114.569633,++
3,Daft Punk - Homework - 09 - Teachers.mp3,123.171456,+++++
4,Erasure - Always.mp3,104.091267,+++
5,Fats Waller - All That Meat and No Potatoes.mp3,141.969927,+
6,Ice Cube - It Was A Good Day.mp3,82.152067,++++
7,J. S. Bach - Glenn Gould - Prelude No. 1 C Maj...,124.576271,++
8,John Cameron - Liquid Sunshine.mp3,133.444537,++
9,Johnny Cash - Ring of Fire.mp3,138.186756,++
